<a href="https://colab.research.google.com/github/AyushK-101/Hamoye-Fall/blob/main/StageB/StageB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd


In [4]:
df=pd.read_csv("/content/energydata_complete.csv")
df.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19735 entries, 0 to 19734
Data columns (total 29 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   date         19735 non-null  object 
 1   Appliances   19735 non-null  int64  
 2   lights       19735 non-null  int64  
 3   T1           19735 non-null  float64
 4   RH_1         19735 non-null  float64
 5   T2           19735 non-null  float64
 6   RH_2         19735 non-null  float64
 7   T3           19735 non-null  float64
 8   RH_3         19735 non-null  float64
 9   T4           19735 non-null  float64
 10  RH_4         19735 non-null  float64
 11  T5           19735 non-null  float64
 12  RH_5         19735 non-null  float64
 13  T6           19735 non-null  float64
 14  RH_6         19735 non-null  float64
 15  T7           19735 non-null  float64
 16  RH_7         19735 non-null  float64
 17  T8           19735 non-null  float64
 18  RH_8         19735 non-null  float64
 19  T9  

There are no missing values

In [7]:
df = df.drop(['date', 'lights'], axis = 1) 


Dropping Date and Lights

In [10]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

dfs = pd.DataFrame(scaler.fit_transform(df),  columns = df.columns)

X = dfs.drop(columns=['Appliances'])
y = dfs['Appliances']

Utilizing MaxMinScaler

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

regdf = dfs[['T2', 'T6']]

regdf.head()

,T2,T6
0,0.225345,0.381070
1,0.225345,0.375443
2,0.225345,0.367487
3,0.225345,0.363800
4,0.225345,0.361859


In [12]:
x= regdf['T2'].values.reshape(-1,1)
y = regdf['T6'].values.reshape(-1,1)

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.3, random_state=42)
from sklearn import linear_model


linreg = linear_model.LinearRegression()
linreg.fit(xtrain, ytrain)


pred = linreg.predict(xtest)
from sklearn.metrics import r2_score

r2_score = r2_score(ytest, pred)
print('r-square:',(round(r2_score, 2)))

r-square: 0.64


Calculating R^2

In [15]:
from sklearn import linear_model

regr = linear_model.LinearRegression()

regr.fit(X_train, y_train)

energy_pred = regr.predict(X_test)

from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, energy_pred)

print('mae:',(round(mae, 2)))

import numpy as np
rss = np.sum(np.square(y_test - energy_pred))
print('rss:',(round(rss, 2)))

from sklearn.metrics import  mean_squared_error

rmse = np.sqrt(mean_squared_error(y_test, energy_pred))
print('rsme:',(round(rmse, 3)))

from sklearn.metrics import r2_score

r2_score = r2_score(y_test, energy_pred)
print('r2:',(round(r2_score, 2)))

mae: 0.05
rss: 45.35
rsme: 0.088
r2: 0.15


Calculating other metrics

In [18]:
def wtcalc(model, feat, col_name):
    
  
    weights = pd.Series(model.coef_, feat.columns).sort_values()
    weights_df = pd.DataFrame(weights).reset_index()
    weights_df.columns = ['feature', col_name]
    weights_df[col_name].round(3)
    
    return weights_df

linearmodel_wts = wtcalc(regr, X_train, 'model_Wt')
linearmodel_wts

,feature,model_Wt
0,RH_2,-0.456698
1,T_out,-0.321860
2,T2,-0.236178
3,T9,-0.189941
4,RH_8,-0.157595
5,RH_out,-0.077671
6,RH_7,-0.044614
7,RH_9,-0.039800
8,T5,-0.015657
9,T1,-0.003281


In [19]:
from sklearn.linear_model import Ridge

ridge_reg = Ridge(alpha=0.4)
ridge_reg.fit(X_train, y_train)

ridge_pred = ridge_reg.predict(X_test)

from sklearn.metrics import  mean_squared_error

rmse = np.sqrt(mean_squared_error(y_test, ridge_pred))
print('rsme:',(round(rmse, 3)))

rsme: 0.088


No change is RSME

In [22]:
from sklearn.linear_model import Lasso

lasso_reg = Lasso(alpha=0.001)
lasso_reg.fit(X_train, y_train)

lasso_pred = lasso_reg.predict(X_test)

Lasso Regression

In [23]:
lassowtsdf = wtcalc(lasso_reg, X_train, 'Lasso_weight')
lassowtsdf

,feature,Lasso_weight
0,RH_out,-0.049557
1,RH_8,-0.000110
2,T1,0.000000
3,Tdewpoint,0.000000
4,Visibility,0.000000
5,Press_mm_hg,-0.000000
6,T_out,0.000000
7,RH_9,-0.000000
8,T9,-0.000000
9,T8,0.000000


In [24]:
from sklearn.metrics import  mean_squared_error

rmse = np.sqrt(mean_squared_error(y_test, lasso_pred))
print('rsme:',(round(rmse, 3)))

rsme: 0.094
